Pattern -> Descriptors -> Embeddings -> Coordinates (by patt index)

  -->  MIDI Drum Assignment -> Notes In Channels -> Flattening Type Results (by patt index) ==> for all flattening types.
  
---
Build and test NN for each (flat, coord) combo & report MAE results.

In [1]:
## Imports ##
import os
import numpy as np
import sys
import random
import importlib

## TODO: double check these need to be imported in this file
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as tdata
import torch.backends.mps
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [28]:
## File Locations (GitHub repo dir) ##
dir=os.getcwd()
os.listdir(dir)

import descriptors as desc
import functions as func
import flatten as flat
import neuralnetwork

In [3]:
## File Locations (Google Drive) ##
#drive.mount('/content/drive/MyDrive/Tapping_Force_Experiment_Data')

'''
Mounting my drive doesn't work by code, but does by side panel.
We will need to figure this

dir = os.chdir('/content/drive/MyDrive/Tapping_Force_Experiment_Data')
os.listdir(dir)
import descriptors as desc
import functions as func
'''

"\nMounting my drive doesn't work by code, but does by side panel.\nWe will need to figure this\n\ndir = os.chdir('/content/drive/MyDrive/Tapping_Force_Experiment_Data')\nos.listdir(dir)\nimport descriptors as desc\nimport functions as func\n"

In [4]:
## Patterns ##
all_pattlists, all_names = func.rootfolder2pattlists("midi/","allinstruments")


In [21]:
## Descriptors & Embedding Coordinates ~23s ##
d = desc.lopl2descriptors(all_pattlists)

mds_pos = func.d_mtx_2_mds(d)

In [17]:
## Convert Patterns (Full MIDI --> (8?) 3-Channel) ##
patterns = [] # 3 channel patterns
importlib.reload(flat)

for patt in all_pattlists:
    p = flat.get_LMH(patt)
    patterns.append(p)
print(patterns[1])

[[1], [], [2], [], [1], [], [2], [], [1], [], [2], [], [1], [], [2], []]


### Analytical Flattening Types:

Types: | Combo: | FuncRef: | Index: |
|---|:---:|---|---:|
**Onset Density** | (only) | [0,0,0] | 0 |
&emsp; &emsp; +| Metrical Strength  | [0,1,0] | 1 |
&emsp; &emsp; +| Syncopation  | [0,0,1] | 2 |
&emsp; &emsp; +| Metrical Strength <br>& Syncopation  | [0,1,1] | 3 |
--- | --- | --- | ---
**Frequency Weighted <br>Onset Density** | (only) | [1,0,0] | 4 |
&emsp; &emsp; +| Metrical Strength  | [1,1,0] | 5 |
&emsp; &emsp; +| Syncopation  | [1,0,1] | 6 |
&emsp; &emsp; +| Metrical Strength <br>& Syncopation  | [1,1,1] | 7 |
--- | --- | --- | ---
**Pattern Relative <br>Onset Density** | (only) | [2,0,0] | 8 |
&emsp; &emsp; +| Metrical Strength  | [2,1,0] | 9 |
&emsp; &emsp; +| Syncopation  | [2,0,1] | 10 |
&emsp; &emsp; +| Metrical Strength <br>& Syncopation  | [2,1,1] | 11 |
--- | --- | --- | ---
**Note Presence** | (only/mono) | [3,0,0] | 12 |
&emsp; &emsp; +| Metrical Strength  | [3,1,0] | 13 |
&emsp; &emsp; +| Syncopation  | [3,0,1] | 14 |
&emsp; &emsp; +| Metrical Strength <br>& Syncopation  | [3,1,1] | 15 |
&emsp; &emsp; +| Polyphonic Syncopation  | [3,0,2] | 16 |




In [13]:
# density types
onset_density=0
freq_weighted_onset_density=1
relative_onset_density=2
note_presence=3
# meter
no_meter=0
meter=1
# sync
no_sync=0
sync=1
polysync=2

In [ ]:
importlib.reload(flat)
## Convert 3-channel to all 17 flat types. ##

n_patterns = len(patterns)
types = 17
flat_names = ['OnsetDen', 'OnsetDen_Meter', 'OnsetDen_Sync','OnsetDen_Sync_Meter',
              'FreqWeightOnsetDen', 'FreqWeightOnsetDen_Meter', 'FreqWeightOnsetDen_Sync','FreqWeightOnsetDen_Sync_Meter',
              'RelOnsetDen', 'RelOnsetDen_Meter', 'RelOnsetDen_Sync','RelOnsetDen_Sync_Meter',
              'Mono', 'Mono_Meter', 'Mono_Sync','Mono_Sync_Meter', 'Mono_PolySync']
len_patt = 16

# test
# patterns=patterns[13:20]

flatterns = np.array([[[0.0 for z in range(len_patt)] for y in range(types)] for x in range(n_patterns)], dtype=float) 
for p in range(len(patterns)):
    pattern = patterns[p]
# Onset Density
    flatterns[p][0] = flat.flat(pattern, onset_density, no_meter, no_sync)
    flatterns[p][1] = flat.flat(pattern, onset_density, meter, no_sync)
    flatterns[p][2] = flat.flat(pattern, onset_density, no_meter, sync)
    flatterns[p][3] = flat.flat(pattern, onset_density, meter, sync)
# Frequency Weighted Onset Density
    flatterns[p][4] = flat.flat(pattern, freq_weighted_onset_density, no_meter, no_sync)
    flatterns[p][5] = flat.flat(pattern, freq_weighted_onset_density, meter, no_sync)
    flatterns[p][6] = flat.flat(pattern, freq_weighted_onset_density, no_meter, sync)
    flatterns[p][7] = flat.flat(pattern, freq_weighted_onset_density, meter, sync)
# Pattern Relatiive Onset Density
    flatterns[p][8] = flat.flat(pattern, relative_onset_density, no_meter, no_sync)
    flatterns[p][9] = flat.flat(pattern, relative_onset_density, meter, no_sync)
    flatterns[p][10] = flat.flat(pattern, relative_onset_density, no_meter, sync)
    flatterns[p][11] = flat.flat(pattern, relative_onset_density, meter, sync)
# Note Presence
    flatterns[p][12] = flat.flat(pattern, note_presence, no_meter, no_sync)
    flatterns[p][13] = flat.flat(pattern, note_presence, meter, no_sync)
    flatterns[p][14] = flat.flat(pattern, note_presence, no_meter, sync)
    flatterns[p][15] = flat.flat(pattern, note_presence, meter, sync)
    flatterns[p][16] = flat.flat(pattern, note_presence, no_meter, polysync)
    if p %100==0:
        print(f"[x] - {p}/{len(patterns)}")
print(f"[x] - {p+1}/{len(patterns)}")

In [ ]:
''' CHANGE THIS TO DESIRED FLATTENING TYPE'''
flat_type = 17

""" print(flatterns.shape)
flattern = flatterns[:,flat_type,:]
print(flat_names[flat_type])
print(np.round(flattern[13:19],4)) """

for i in range(flat_type):
    print(flatterns.shape)
    flattern = flatterns[:,i,:]
    print(flat_names[i])
    print(flattern[824]) # example

In [102]:
## Load? ##
_load = False
model_dir=''
importlib.reload(neuralnetwork)

## Define model structure ##
model_layers=4
len_pattern=16 # input layer
firstlayer=32
secondlayer=32
thirdlayer=16
fourthlayer=16
coords=2 # output layer

## Build / Load model ##
if _load:
    model=neuralnetwork.load_model(model_dir)
else:
    model = neuralnetwork.build_model(len_pattern=16,firstlayer=32,secondlayer=32,thirdlayer=16,fourthlayer=16,coords=2)

## Select model hyperparameters ##
learning_rate = 0.002
batch_size = 32
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
epochs = 300

train_loss = 0.0
test_loss = 0.0

Using cpu device


### Analytical Flattening Types:

Types: | Combo: | FuncRef: | Index: |
|---|:---:|---|---:|
**Onset Density** | (only) | [0,0,0] | 0 |
&emsp; &emsp; +| Metrical Strength  | [0,1,0] | 1 |
&emsp; &emsp; +| Syncopation  | [0,0,1] | 2 |
&emsp; &emsp; +| Metrical Strength <br>& Syncopation  | [0,1,1] | 3 |
--- | --- | --- | ---
**Frequency Weighted <br>Onset Density** | (only) | [1,0,0] | 4 |
&emsp; &emsp; +| Metrical Strength  | [1,1,0] | 5 |
&emsp; &emsp; +| Syncopation  | [1,0,1] | 6 |
&emsp; &emsp; +| Metrical Strength <br>& Syncopation  | [1,1,1] | 7 |
--- | --- | --- | ---
**Pattern Relative <br>Onset Density** | (only) | [2,0,0] | 8 |
&emsp; &emsp; +| Metrical Strength  | [2,1,0] | 9 |
&emsp; &emsp; +| Syncopation  | [2,0,1] | 10 |
&emsp; &emsp; +| Metrical Strength <br>& Syncopation  | [2,1,1] | 11 |
--- | --- | --- | ---
**Note Presence** | (only/mono) | [3,0,0] | 12 |
&emsp; &emsp; +| Metrical Strength  | [3,1,0] | 13 |
&emsp; &emsp; +| Syncopation  | [3,0,1] | 14 |
&emsp; &emsp; +| Metrical Strength <br>& Syncopation  | [3,1,1] | 15 |
&emsp; &emsp; +| Polyphonic Syncopation  | [3,0,2] | 16 |



In [100]:
### SELECT WHICH ALG ###
flattern_num = 5

input_data = np.asarray(flatterns[:,flattern_num,:],dtype=np.float32)
target_data = np.asarray(mds_pos,dtype=np.float32)


# make semi continuous
_semicontinuous = True
if _semicontinuous:
    p_means = np.mean(input_data, axis=1, keepdims=True)
    input_data[input_data<p_means] = 0.0

# make discrete
_discrete = False
if _discrete:
    p_means = np.mean(input_data, axis=1, keepdims=True)
    input_data[input_data>=p_means] = 1
    input_data[input_data<p_means] = 0

print(input_data[220:240])

[[1.         0.         0.         0.         0.3351724  0.
  0.         0.         0.22951724 0.         0.         0.
  0.3351724  0.         0.         0.        ]
 [1.         0.         0.         0.         0.3469091  0.
  0.         0.         0.512      0.         0.         0.
  0.3469091  0.         0.         0.        ]
 [1.         0.         0.         0.         0.3351724  0.
  0.         0.         0.22951724 0.         0.         0.
  0.3351724  0.         0.         0.        ]
 [1.         0.         0.         0.         0.3469091  0.
  0.         0.         0.512      0.         0.         0.
  0.3469091  0.         0.         0.        ]
 [1.         0.         0.         0.         0.3351724  0.
  0.         0.         0.22951724 0.         0.         0.
  0.3351724  0.         0.         0.        ]
 [1.         0.         0.         0.         0.3469091  0.
  0.         0.         0.512      0.         0.         0.
  0.3469091  0.         0.         0.        

In [91]:
## 10 fold cross evaluation splits ##
training_DL_list = []
validation_DL_list = []

seed=np.random.randint(0,1000) ## for traintest splits. 
train, test, train_coords, test_coords = train_test_split(input_data, target_data, test_size=0.2, random_state=seed)
test_dataset = neuralnetwork.CustomDataset(test, test_coords)
test_dataloader = tdata.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# make train and validation splits on training data
folds=10
kfold = KFold(n_splits=folds, shuffle=True) 

for train_idx, validation_idx in kfold.split(train):
    
    train_fold, validation_fold = input_data[train_idx], input_data[validation_idx]
    target_fold, target_val_fold = target_data[train_idx], target_data[validation_idx]

    train_dataset = neuralnetwork.CustomDataset(train_fold, target_fold)
    validation_dataset = neuralnetwork.CustomDataset(validation_fold, target_val_fold)

    train_dataloader = tdata.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    validation_dataloader = tdata.DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

    training_DL_list.append(train_dataloader)
    validation_DL_list.append(validation_dataloader)

In [101]:
## Train a model on 10-folds for one flattening algorithm ##
## This process can take around 35-40sec ##
importlib.reload(neuralnetwork)
accuracy=[[0.0,0.0] for n in range(len(training_DL_list))] # num splits
for split in range(len(training_DL_list)):
    dataloader = training_DL_list[split]
    val_dataloader = validation_DL_list[split]

    neuralnetwork.train_loop(model, dataloader, epochs, criterion, optimizer)
    all_acc = []
    all_acc, accuracy[split][1] = neuralnetwork.test_loop(model, val_dataloader, criterion)
    accuracy[split][0] = np.average(all_acc)
    print(f"Fold:{split} Avg:{accuracy[split][0]:.4f}({np.std(all_acc):.4f}) FinalTestLoss:{accuracy[split][1]:.3f}")
print(f"{np.average(accuracy[split][0])}")


Using cpu device
Fold:0 Avg:0.1377(0.1372) FinalTestLoss:0.017
Fold:1 Avg:0.1365(0.1122) FinalTestLoss:0.016
Fold:2 Avg:0.1186(0.1206) FinalTestLoss:0.013
Fold:3 Avg:0.1073(0.1035) FinalTestLoss:0.010
Fold:4 Avg:0.1051(0.1051) FinalTestLoss:0.011
Fold:5 Avg:0.0978(0.0917) FinalTestLoss:0.009
Fold:6 Avg:0.0909(0.1109) FinalTestLoss:0.012
Fold:7 Avg:0.1098(0.1320) FinalTestLoss:0.015
Fold:8 Avg:0.1097(0.1305) FinalTestLoss:0.014
Fold:9 Avg:0.0998(0.1282) FinalTestLoss:0.015
0.0998165130623829


In [98]:
## Evaluate Model ##
mean_abs_errors, _ = neuralnetwork.test_loop(model, test_dataloader, criterion)
mean_abs_error = np.average(mean_abs_errors)
mean_CV = round(np.std(mean_abs_errors)/mean_abs_error,4)
print(f"Algorithm: {flat_names[flattern_num]} --> MAE: {mean_abs_error:.4f} [std:{np.std(mean_abs_errors)}] ***** Test Dataset Only *****")

whole_dataset = neuralnetwork.CustomDataset(input_data, target_data)
whole_dataloader = tdata.DataLoader(whole_dataset, batch_size=batch_size, shuffle=False)
mean_abs_errors, _ = neuralnetwork.test_loop(model, whole_dataloader, criterion)
mean_abs_error = np.average(mean_abs_errors)
mean_CV = round(np.std(mean_abs_errors)/mean_abs_error,4)
print(f"Algorithm: {flat_names[flattern_num]} --> MAE: {mean_abs_error:.4f} [std:{np.std(mean_abs_errors)}] ***** Whole Dataset *****")

Algorithm: FreqWeightOnsetDen_Meter --> MAE: 0.1037 [std:0.16062075116934466] ***** Test Dataset Only *****
Algorithm: FreqWeightOnsetDen_Meter --> MAE: 0.1137 [std:0.157566096145877] ***** Whole Dataset *****


The code below will build models for all flattening algorithms, but it will take time to complete. 

In [150]:
## For all Flat: NN build + (train + test)-10fold ## WARNING: LONGTIME!
# TODO
importlib.reload(neuralnetwork)

train_scores = []
test_scores = []

for flat in range(len(flat_names)):
    print(f"Moving on to flat-type: {flat_names[flat]}" )
    ## PREPARE ----------------------------------------
    model = neuralnetwork.build_model(len_pattern=16,firstlayer=32,secondlayer=32,thirdlayer=16,fourthlayer=16,coords=2)
    ## Select model hyperparameters ##
    learning_rate = 0.0015
    batch_size = 32
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    epochs = 300

    input_data = np.asarray(flatterns[:,flat,:],dtype=np.float32)
    target_data = np.asarray(mds_pos,dtype=np.float32)

    # make semi continuous
    _semicontinuous = False
    if _semicontinuous:
        p_means = np.mean(input_data, axis=1, keepdims=True)
        input_data[input_data<p_means] = 0.0

    # make discrete
    _discrete = True
    if _discrete:
        p_means = np.mean(input_data, axis=1, keepdims=True)
        input_data[input_data>=p_means] = 1.0
        input_data[input_data<p_means] = 0.0

    ## 10 fold cross evaluation splits ##
    training_DL_list = []
    validation_DL_list = []

    seed=np.random.randint(0,1000) ## for traintest splits. 
    train, test, train_coords, test_coords = train_test_split(input_data, target_data, test_size=0.2, random_state=seed)
    test_dataset = neuralnetwork.CustomDataset(test, test_coords)
    test_dataloader = tdata.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # make train and validation splits on training data
    folds=5
    kfold = KFold(n_splits=folds, shuffle=True) 

    for train_idx, validation_idx in kfold.split(train):
        
        train_fold, validation_fold = input_data[train_idx], input_data[validation_idx]
        target_fold, target_val_fold = target_data[train_idx], target_data[validation_idx]

        train_dataset = neuralnetwork.CustomDataset(train_fold, target_fold)
        validation_dataset = neuralnetwork.CustomDataset(validation_fold, target_val_fold)

        train_dataloader = tdata.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        validation_dataloader = tdata.DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

        training_DL_list.append(train_dataloader)
        validation_DL_list.append(validation_dataloader)

    accuracy=[[0.0,0.0] for n in range(len(training_DL_list))] # num splits
    for split in range(len(training_DL_list)):
        dataloader = training_DL_list[split]
        val_dataloader = validation_DL_list[split]

        neuralnetwork.train_loop(model, dataloader, epochs, criterion, optimizer)
        all_acc = []
        all_acc, accuracy[split][1] = neuralnetwork.test_loop(model, val_dataloader, criterion)
        accuracy[split][0] = np.average(all_acc)
        print(f"Fold:{split} Avg:{accuracy[split][0]:.4f} FinalTestLoss:{accuracy[split][1]:.3f}")
        train_scores.append(all_acc)

    ## Evaluate Model ##
    mean_abs_errors, _ = neuralnetwork.test_loop(model, test_dataloader, criterion)
    mean_abs_error = np.average(mean_abs_errors)
    mean_CV = round(np.std(mean_abs_errors)/mean_abs_error,4)
    print(f"Algorithm: {flat_names[flat]} --> MAE: {mean_abs_error:.4f} [std:{np.std(mean_abs_errors)}] ***** Test Dataset Only *****")

    test_scores.append(mean_abs_errors)
    
    whole_dataset = neuralnetwork.CustomDataset(input_data, target_data)
    whole_dataloader = tdata.DataLoader(whole_dataset, batch_size=batch_size, shuffle=False)
    mean_abs_errors, _ = neuralnetwork.test_loop(model, whole_dataloader, criterion)
    mean_abs_error = np.average(mean_abs_errors)
    mean_CV = round(np.std(mean_abs_errors)/mean_abs_error,4)
    print(f"Algorithm: {flat_names[flat]} --> MAE: {mean_abs_error:.4f} [std:{np.std(mean_abs_errors)}] ***** Whole Dataset *****")
    


Using cpu device
Moving on to flat-type: OnsetDen
Fold:0 Avg:0.1628 FinalTestLoss:0.023
Fold:1 Avg:0.1425 FinalTestLoss:0.017
Fold:2 Avg:0.1496 FinalTestLoss:0.021
Fold:3 Avg:0.1552 FinalTestLoss:0.022
Fold:4 Avg:0.1324 FinalTestLoss:0.017
Algorithm: OnsetDen --> MAE: 0.1579 [std:0.13659816229333624] ***** Test Dataset Only *****
Algorithm: OnsetDen --> MAE: 0.1535 [std:0.13003744486214106] ***** Whole Dataset *****
Moving on to flat-type: OnsetDen_Meter
Fold:0 Avg:0.2162 FinalTestLoss:0.033
Fold:1 Avg:0.2169 FinalTestLoss:0.031
Fold:2 Avg:0.1996 FinalTestLoss:0.025
Fold:3 Avg:0.1987 FinalTestLoss:0.027
Fold:4 Avg:0.2128 FinalTestLoss:0.030
Algorithm: OnsetDen_Meter --> MAE: 0.2265 [std:0.1197344374683496] ***** Test Dataset Only *****
Algorithm: OnsetDen_Meter --> MAE: 0.2128 [std:0.1155751288122925] ***** Whole Dataset *****
Moving on to flat-type: OnsetDen_Sync
Fold:0 Avg:0.0636 FinalTestLoss:0.006
Fold:1 Avg:0.0580 FinalTestLoss:0.004
Fold:2 Avg:0.0538 FinalTestLoss:0.004
Fold:3 Av

In [151]:
# some data rearranging
trn_scores = []
train_means = np.array([[0.0 for x in range(folds)] for y in range(len(flat_names))], dtype=float)
train_stds = np.array([[0.0 for x in range(folds)] for y in range(len(flat_names))], dtype=float)
for f in range(len(flat_names)):
    t_s = []
    for g in range(folds):
        t_s.append(train_scores[f*g+g])
        train_means[f][g]=np.mean(np.asarray(train_scores[f*g+g],dtype=float))
        train_stds[f][g]=np.std(np.asarray(train_scores[f*g+g],dtype=float))
    trn_scores.append(t_s)

for a in range(len(flat_names)):
    print(f"{flat_names[a]}")
    train_mean = np.mean(train_means[a])
    train_std = np.mean(train_stds[a])
    #print(f"TRAIN: {train_mean:.4f}, {train_std:.4f}")
    print(f"TEST: {np.mean(test_scores[a]):.4f}, ({np.std(test_scores[a]):.4f})")

OnsetDen
TEST: 0.1579, (0.1366)
OnsetDen_Meter
TEST: 0.2265, (0.1197)
OnsetDen_Sync
TEST: 0.0668, (0.0873)
OnsetDen_Sync_Meter
TEST: 0.1527, (0.1258)
FreqWeightOnsetDen
TEST: 0.1526, (0.1350)
FreqWeightOnsetDen_Meter
TEST: 0.2224, (0.1046)
FreqWeightOnsetDen_Sync
TEST: 0.1552, (0.1393)
FreqWeightOnsetDen_Sync_Meter
TEST: 0.2334, (0.1176)
RelOnsetDen
TEST: 0.1426, (0.1238)
RelOnsetDen_Meter
TEST: 0.1843, (0.1134)
RelOnsetDen_Sync
TEST: 0.1451, (0.1356)
RelOnsetDen_Sync_Meter
TEST: 0.1582, (0.1255)
Mono
TEST: 0.0976, (0.0980)
Mono_Meter
TEST: 0.2116, (0.1060)
Mono_Sync
TEST: 0.1041, (0.0981)
Mono_Sync_Meter
TEST: 0.2145, (0.1013)
Mono_PolySync
TEST: 0.0850, (0.1127)


### 300 0.0015 cont ###

OnsetDen
TEST: 0.0436, (0.0778)

OnsetDen_Meter
TEST: 0.0438, (0.0934)

OnsetDen_Sync
TEST: 0.0444, (0.0878)

OnsetDen_Sync_Meter
TEST: 0.0366, (0.0802)

FreqWeightOnsetDen
TEST: 0.0391, (0.0906)

FreqWeightOnsetDen_Meter
TEST: 0.0517, (0.0838)

FreqWeightOnsetDen_Sync
TEST: 0.0480, (0.0954)

FreqWeightOnsetDen_Sync_Meter
TEST: 0.0539, (0.0923)

RelOnsetDen
TEST: 0.0448, (0.0752)

RelOnsetDen_Meter
TEST: 0.0406, (0.0784)

RelOnsetDen_Sync
TEST: 0.0560, (0.1062)

RelOnsetDen_Sync_Meter
TEST: 0.0543, (0.1111)

Mono
TEST: 0.1042, (0.1010)

Mono_Meter
TEST: 0.1125, (0.1045)

Mono_Sync
TEST: 0.0971, (0.0937)

Mono_Sync_Meter
TEST: 0.0970, (0.0966)

Mono_PolySync
TEST: 0.0672, (0.1084)


### 300 0.0015 semi-cont ###

OnsetDen
TEST: 0.0832, (0.1357)

OnsetDen_Meter
TEST: 0.1085, (0.1317)

OnsetDen_Sync
TEST: 0.0526, (0.1068)

OnsetDen_Sync_Meter
TEST: 0.0777, (0.1180)

FreqWeightOnsetDen
TEST: 0.0585, (0.1041)

FreqWeightOnsetDen_Meter
TEST: 0.1656, (0.1219)

FreqWeightOnsetDen_Sync
TEST: 0.0639, (0.1156)

FreqWeightOnsetDen_Sync_Meter
TEST: 0.1360, (0.1623)

RelOnsetDen
TEST: 0.0808, (0.1217)

RelOnsetDen_Meter
TEST: 0.1211, (0.1304)

RelOnsetDen_Sync
TEST: 0.0739, (0.1390)

RelOnsetDen_Sync_Meter
TEST: 0.0714, (0.1393)

Mono
TEST: 0.1115, (0.1048)

Mono_Meter
TEST: 0.2247, (0.1064)

Mono_Sync
TEST: 0.1081, (0.1081)

Mono_Sync_Meter
TEST: 0.2142, (0.1151)

Mono_PolySync
TEST: 0.0749, (0.1132)

### 300 0.0015 discrete ###

OnsetDen
TEST: 0.1579, (0.1366)

OnsetDen_Meter
TEST: 0.2265, (0.1197)

OnsetDen_Sync
TEST: 0.0668, (0.0873)

OnsetDen_Sync_Meter
TEST: 0.1527, (0.1258)

FreqWeightOnsetDen
TEST: 0.1526, (0.1350)

FreqWeightOnsetDen_Meter
TEST: 0.2224, (0.1046)

FreqWeightOnsetDen_Sync
TEST: 0.1552, (0.1393)

FreqWeightOnsetDen_Sync_Meter
TEST: 0.2334, (0.1176)

RelOnsetDen
TEST: 0.1426, (0.1238)

RelOnsetDen_Meter
TEST: 0.1843, (0.1134)

RelOnsetDen_Sync
TEST: 0.1451, (0.1356)

RelOnsetDen_Sync_Meter
TEST: 0.1582, (0.1255)

Mono
TEST: 0.0976, (0.0980)

Mono_Meter
TEST: 0.2116, (0.1060)

Mono_Sync
TEST: 0.1041, (0.0981)

Mono_Sync_Meter
TEST: 0.2145, (0.1013)

Mono_PolySync
TEST: 0.0850, (0.1127)
